In [1]:
import pyspark as ps
from pyspark.sql.types import *
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
# sc=SparkContext('local') this didn't work
# sc = ps.SparkContext('local[4]') this didn't work
sqlContext = ps.SQLContext(sc)

In [2]:
#Load the Yelp business data using the function `.read.json()` from the `SparkSession()` object with input file.
df = sqlContext.read.json('data/yelp_academic_dataset_business.json.gz')
df.printSchema()

root
 |-- attributes: struct (nullable = true)
 |    |-- Accepts Credit Cards: string (nullable = true)
 |    |-- Accepts Insurance: boolean (nullable = true)
 |    |-- Ages Allowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: struct (nullable = true)
 |    |    |-- casual: boolean (nullable = true)
 |    |    |-- classy: boolean (nullable = true)
 |    |    |-- divey: boolean (nullable = true)
 |    |    |-- hipster: boolean (nullable = true)
 |    |    |-- intimate: boolean (nullable = true)
 |    |    |-- romantic: boolean (nullable = true)
 |    |    |-- touristy: boolean (nullable = true)
 |    |    |-- trendy: boolean (nullable = true)
 |    |    |-- upscale: boolean (nullable = true)
 |    |-- Attire: string (nullable = true)
 |    |-- BYOB: boolean (nullable = true)
 |    |-- BYOB/Corkage: string (nullable = true)
 |    |-- By Appointment Only: boolean (nullable = true)
 |    |-- Caters: boolean (nullable = true)
 |    |-- Coat Check

In [3]:
df.show(1)

+--------------------+--------------------+--------------------+-------+--------------------+--------------------+---------+-----------+-----------------+-------------+----+------------+-----+-----+--------+
|          attributes|         business_id|          categories|   city|        full_address|               hours| latitude|  longitude|             name|neighborhoods|open|review_count|stars|state|    type|
+--------------------+--------------------+--------------------+-------+--------------------+--------------------+---------+-----------+-----------------+-------------+----+------------+-----+-----+--------+
|[null,null,null,n...|vcNAWiLM4dR7D2nww...|[Doctors, Health ...|Phoenix|4840 E Indian Sch...|[[17:00,08:00],[1...|33.499313|-111.983758|Eric Goldberg, MD|           []|true|           9|  3.5|   AZ|business|
+--------------------+--------------------+--------------------+-------+--------------------+--------------------+---------+-----------+-----------------+-------------+

In [4]:
df1=df.toPandas()
df1.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,"(None, None, None, None, None, None, None, Non...",vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]",Phoenix,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","((17:00, 08:00), (17:00, 08:00), None, None, (...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,9,3.5,AZ,business
1,"(true, None, None, None, None, None, None, Non...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],Dravosburg,"202 McClure St\nDravosburg, PA 15034","(None, None, None, None, None, None, None)",40.350519,-79.886930,Clancy's Pub,[],True,4,3.5,PA,business
2,"(None, None, None, None, None, None, None, Non...",cE27W9VPgO88Qxe4ol6y_g,"[Active Life, Mini Golf, Golf]",Bethel Park,"1530 Hamilton Rd\nBethel Park, PA 15234","(None, None, None, None, None, None, None)",40.356896,-80.015910,Cool Springs Golf Center,[],False,5,2.5,PA,business
3,"(None, None, None, None, None, None, None, Non...",HZdLhv6COCleJMo7nPl-RA,"[Shopping, Home Services, Internet Service Pro...",Pittsburgh,"301 S Hills Vlg\nPittsburgh, PA 15241","((21:00, 10:00), (21:00, 10:00), (21:00, 10:00...",40.357620,-80.059980,Verizon Wireless,[],True,3,3.5,PA,business
4,"(true, None, None, full_bar, (False, False, Fa...",mVHrayjG3uZ_RLHkLj-AMg,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,"414 Hawkins Ave\nBraddock, PA 15104","((20:00, 10:00), None, (16:00, 10:00), None, (...",40.408735,-79.866351,Emil's Lounge,[],True,11,4.5,PA,business


In [5]:
#Register the dataframe as a temporary table named yelp_business (this will enable us to query the table later using our SparkSession() object).
df.createOrReplaceTempView("yelp_business")
#yelp_business=spark.table("yelp_business_df")
result = spark.sql("SELECT name, city, state, attributes, stars FROM yelp_business LIMIT 10")
result.show()

+--------------------+-----------+-----+--------------------+-----+
|                name|       city|state|          attributes|stars|
+--------------------+-----------+-----+--------------------+-----+
|   Eric Goldberg, MD|    Phoenix|   AZ|[null,null,null,n...|  3.5|
|        Clancy's Pub| Dravosburg|   PA|[true,null,null,n...|  3.5|
|Cool Springs Golf...|Bethel Park|   PA|[null,null,null,n...|  2.5|
|    Verizon Wireless| Pittsburgh|   PA|[null,null,null,n...|  3.5|
|       Emil's Lounge|   Braddock|   PA|[true,null,null,f...|  4.5|
|Alexion's Bar & G...|   Carnegie|   PA|[true,null,null,f...|  4.0|
|Flynn's E W Tire ...|   Carnegie|   PA|[null,null,null,n...|  1.5|
|Forsythe Miniatur...|   Carnegie|   PA|[null,null,null,n...|  4.0|
|Quaker State Cons...|   Carnegie|   PA|[null,null,null,n...|  2.5|
|Kings Family Rest...|   Carnegie|   PA|[true,null,null,n...|  3.5|
+--------------------+-----------+-----+--------------------+-----+



In [107]:
# Write a query or a sequence of transformations that returns the name of entries that fulfill the following conditions:
# Rated at 5 stars
# In the city of Phoenix
# Accepts credit card (Reference the 'Accepts Credit Card' field by  attributes.`Accepts Credit Cards`. NOTE: We are actually looking for the value 'true', not the boolean value True!)
# Contains Restaurants in the categories array. Hint: LATERAL VIEW explode() Hint: In spark, while using filter() or where()
r1=spark.sql("select name FROM yelp_business WHERE city = 'Phoenix' AND stars = 5")
#r1.show()

#using pyspark
from pyspark.sql.functions import col, array_contains
yelp_business_df.filter((col('stars') == 5) &
                        (col('city') == 'Phoenix') &
                        (col('attributes.`Accepts Credit Cards`') == 'true') &
                        (array_contains(col('categories'),'Restaurants'))).select('name').collect()

#using spark.sql
spark.sql("""SELECT DISTINCT name from yelp_business
                   LATERAL VIEW explode(categories) AS category
                   WHERE stars = 5 
                   AND city = 'Phoenix'
                   AND attributes.`Accepts Credit Cards` = 'true'
                   AND category = 'Restaurants'""").collect()

[Row(name='Panini Bread and Grill'),
 Row(name='Gluten Free Creations Bakery'),
 Row(name='Ten Handcrafted American Fare & Spirits'),
 Row(name='One Eighty Q'),
 Row(name='Subway'),
 Row(name='Banh Mi Bistro Vietnamese Eatery'),
 Row(name='Tacos Huicho'),
 Row(name='Helpings Cafe, Market and Catering'),
 Row(name='Sunfare'),
 Row(name="Bertie's Of Arcadia"),
 Row(name='Los Primos Carniceria'),
 Row(name='The Brown Bag'),
 Row(name='Little Miss BBQ'),
 Row(name='Auslers Grill'),
 Row(name="Adela's Italian"),
 Row(name='Coe Casa'),
 Row(name='Altamimi Restutant'),
 Row(name='Las Jicaras Mexican Grill'),
 Row(name='Queen Creek Olive Mill Oils & Olives Biltmore Fashion Park'),
 Row(name='Saffron JAK Original Stonebread Pizzas'),
 Row(name='Couscous Express'),
 Row(name='The Loaded Potato'),
 Row(name='Santos Lucha Libre'),
 Row(name="Mulligan's Restaurant"),
 Row(name='Taqueria El Chino'),
 Row(name="Jimmy John's"),
 Row(name="Lil Cal's"),
 Row(name='Frenchys Caribbean Dogs'),
 Row(name="E

In [95]:
# Load a dataframe users from S3 link ''s3a://sparkdatasets/users.txt' (no credentials needed but if you encounter any problem just use local copy data/users.txt instead) using spark.read.csv with the following parameters: no headers, use separator ";", and infer the schema of the underlying data (for now). Use .show(5) and .printSchema() to check the result.
# Create a schema for this dataset using proper names and types for the columns, using types from the pyspark.sql.types module (see lecture). Use that schema to read the users dataframe again and use .printSchema() to check the result.
# Note: Each row in the users file represents the user with his/her user_id, name, email, phone.

    users_schema = StructType( [
    StructField('id',IntegerType(),True),
    StructField('name',StringType(),True),
    StructField('email',StringType(),True),
    StructField('phone',StringType(),True) ] )
# reading the file using SparkContext
users = spark.read.csv('data/users.txt',
                         header=False,       # use headers or not
                         quote='"',         # char for quotes
                         sep=";",           # char for separation
                         schema=users_schema, # using predefined schema
                         inferSchema=False)  
#rdd = sc.textFile('data/users.txt')
users.take(5)

[Row(id=1106214172, name='Prometheus Barwis', email='prometheus.barwis@me.com', phone='(533) 072-2779'),
 Row(id=527133132, name='Ashraf Bainbridge', email='ashraf.bainbridge@gmail.com', phone=None),
 Row(id=1290614884, name='Alain Hennesey', email='alain.hennesey@facebook.com,alain.hennesey@me.com', phone='(942) 208-8460,(801) 938-2376'),
 Row(id=1700818057, name='Hamed Fingerhuth', email='hamed.fingerhuth@msn.com,hamed.fingerhuth@me.com', phone=None),
 Row(id=17378782, name='Annamae Leyte', email='annamae.leyte@msn.com,annamae.leyte@facebook.com', phone=None)]

In [64]:
transactions_rdd=sc.textFile('data/transactions.txt')
print(type(transactions_rdd))

<class 'pyspark.rdd.RDD'>


In [65]:
transactions_rdd.take(5)

['815581247;$144.82;2015-09-05',
 '1534673027;$140.93;2014-03-11',
 '842468364;$104.26;2014-05-06',
 '1720001139;$194.60;2015-08-24',
 '1397891675;$307.72;2015-09-25']

In [80]:
def casting_function(trans_id, amount, date):
    return(int(trans_id), float(amount), date)

In [90]:
# Load an RDD transactions_rdd from S3 link ''s3a://sparkdatasets/transactions.txt' (no credentials needed but if you encounter any problem just use local copy data/transactions.txt instead) using spark.sparkContext.textFile. Use .take(5) to check the result.
# Use .map() to split those csv-like lines, to strip the dollar sign on the second column, and to cast each column to its proper type.

df_schema = StructType([StructField('id',StringType(),True),StructField('amount',FloatType(),True),StructField('date',StringType(),True)])
rdd_trans = spark.sparkContext.textFile('data/transactions.txt').map(lambda s : s.split(';')).map(lambda row : (int(row[0]),float(row[1].lstrip("$")),row[2]))
#.filter(lambda x: not x[0].startswith('$'))
#.map(lambda x : x.split())
rdd_trans.collect()
rdd_trans.take(5) 

[(815581247, 144.82, '2015-09-05'),
 (1534673027, 140.93, '2014-03-11'),
 (842468364, 104.26, '2014-05-06'),
 (1720001139, 194.6, '2015-08-24'),
 (1397891675, 307.72, '2015-09-25')]

In [77]:
# Create a schema for this dataset using proper names and types for the columns, using types from the pyspark.sql.types module (see lecture). Use that schema to convert transactions_rdd into a dataframe transactions and use .show(5) and .printSchema() to check the result.
# Each row in the transactions file has the columns  user_id, amount_paid, date.
schema = StructType([StructField('id',StringType(),True),StructField('amount',FloatType(),True),StructField('date',StringType(),True)])

In [91]:
#df_trans= sqlContext.createDataFrame(rdd_trans,schema)
df_trans = sqlContext.createDataFrame(rdd_trans, schema)
df_trans.printSchema()
df_trans.show()

root
 |-- id: string (nullable = true)
 |-- amount: float (nullable = true)
 |-- date: string (nullable = true)

+----------+------+----------+
|        id|amount|      date|
+----------+------+----------+
| 815581247|144.82|2015-09-05|
|1534673027|140.93|2014-03-11|
| 842468364|104.26|2014-05-06|
|1720001139| 194.6|2015-08-24|
|1397891675|307.72|2015-09-25|
| 926282663| 36.69|2014-10-24|
| 694853136| 39.59|2014-11-26|
| 636287877|430.94|2015-06-12|
|1396310477|  31.4|2014-12-05|
|1279939289|180.69|2015-03-26|
| 859061953|383.35|2014-06-06|
|1983919868| 256.2|2015-09-28|
| 589339046|930.56|2014-09-21|
|1559785598|423.77|2015-05-18|
| 347589978|309.53|2015-10-11|
| 963722938|299.19|2014-04-06|
|1808365853|426.21|2015-09-10|
| 417552135|732.27|2015-09-30|
| 744965566|186.33|2015-12-30|
|1513020241| 925.8|2014-10-06|
+----------+------+----------+
only showing top 20 rows



In [105]:
#Write a sequence of transformations or a SQL query that returns the names and the amount paid for the users with the top 10 transaction amounts.
users.createOrReplaceTempView('users')
df_trans.createOrReplaceTempView('transactions')

r1=spark.sql("""select users.name, trans.amount FROM 
             (SELECT * FROM transactions ORDER BY amount DESC LIMIT 10) trans
             JOIN users
             ON users.id = trans.id""").collect()
r1

[Row(name='Raziel Merk', amount=999.989990234375),
 Row(name='Andrian Waite', amount=999.989990234375),
 Row(name='Kianu Dyneley', amount=999.989990234375),
 Row(name='Landri Fulshur', amount=999.989990234375),
 Row(name='Leilani Cranstoun', amount=999.97998046875),
 Row(name='Samyrah Milbourne', amount=999.97998046875),
 Row(name='Vishwak Farrow', amount=999.97998046875),
 Row(name='Veida Hubbard', amount=999.97998046875),
 Row(name='Ori Horrage', amount=999.97998046875),
 Row(name='Zasia Scrivens', amount=999.97998046875)]